In [2]:
from datasets import load_dataset

dataset = load_dataset("OpenAssistant/oasst1")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/84437 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/4401 [00:00<?, ? examples/s]

Dataset parquet downloaded and prepared to C:/Users/mathew.kenny/.cache/huggingface/datasets/OpenAssistant___parquet/OpenAssistant--oasst1-2960c57d7e52ab15/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
train_dataset = dataset['train']      # len(train)=84437 (95%)
val_dataset = dataset['validation']   # len(val)=4401 (5%)

In [6]:
train

Dataset({
    features: ['message_id', 'parent_id', 'user_id', 'created_date', 'text', 'role', 'lang', 'review_count', 'review_result', 'deleted', 'rank', 'synthetic', 'model_name', 'detoxify', 'message_tree_id', 'tree_state', 'emojis', 'labels'],
    num_rows: 84437
})

In [79]:
# Creating a DataFrame with the keys 'text', 'parent_id', 'message_id' and 'role'
df = pd.DataFrame({'message_tree_id': train_dataset['message_tree_id'], 'parent_id': train_dataset['parent_id'], 'message_id': train_dataset['message_id'], 'role': train_dataset['role'], 'text': train_dataset['text']})
df

,message_tree_id,parent_id,message_id,role,text
0,6ab24d72-0181-4594-a9cd-deaf170242fb,None,6ab24d72-0181-4594-a9cd-deaf170242fb,prompter,Can you write a short introduction about the r...
1,6ab24d72-0181-4594-a9cd-deaf170242fb,6ab24d72-0181-4594-a9cd-deaf170242fb,c8e83833-ecbc-44fe-b6db-735228c25a1c,assistant,"""Monopsony"" refers to a market structure where..."
2,6ab24d72-0181-4594-a9cd-deaf170242fb,c8e83833-ecbc-44fe-b6db-735228c25a1c,6708c47f-05c9-4346-b3d2-40b2bd24fde4,prompter,Now explain it to a dog
3,6ab24d72-0181-4594-a9cd-deaf170242fb,6ab24d72-0181-4594-a9cd-deaf170242fb,343ee2d4-87ae-41fd-a768-bdd65959dc4a,assistant,Monopsony is a market structure in which there...
4,6ab24d72-0181-4594-a9cd-deaf170242fb,343ee2d4-87ae-41fd-a768-bdd65959dc4a,18145bf4-37fd-4ac0-80f5-6108b5f2b365,prompter,How can one fight back when a monospony had be...
...,...,...,...,...,...
84432,2e934881-260c-4a6c-8f09-66e0312f84ac,2e934881-260c-4a6c-8f09-66e0312f84ac,47ba67f2-f711-4feb-91ff-0a0ffe556fe5,assistant,James Potter eta Lily Potter ziren Harry Potte...
84433,d3946880-4402-42a8-8a9e-6dab5efa787f,None,d3946880-4402-42a8-8a9e-6dab5efa787f,prompter,Nork jarri zion ahotsa Dragoi Bolako Vegetari ...
84434,d3946880-4402-42a8-8a9e-6dab5efa787f,d3946880-4402-42a8-8a9e-6dab5efa787f,4e8a349d-7781-42ae-8868-1ecf5df4c2cb,assistant,Dragoi bolaren aurreneko denboraldian ez da Ve...
84435,d3946880-4402-42a8-8a9e-6dab5efa787f,d3946880-4402-42a8-8a9e-6dab5efa787f,ca45d4bd-2da7-4d56-beec-bf9f772e8426,assistant,Dragoi Bolako Vegetari euskarazko ahotsa jarri...


In [75]:
# Creating a new DF to store the prompts and responses
output_df = pd.DataFrame(columns=['prompt', 'response'])

# Collecting list of unqiue tree ids
tree_ids = set(df['message_tree_id'].unique())

# Looping to collect each prompt and response
for tree_id in tree_ids:
    temp_df = df[df['message_tree_id'] == tree_id]
    prompt_df = temp_df[temp_df["role"] == 'prompter']
    assistant_df = temp_df[temp_df["role"] == 'assistant']

    prompt_message_ids = prompt_df['message_id'].tolist()

    prompts = []
    responses = []

    for id in prompt_message_ids:
        prompt = prompt_df[prompt_df['message_id'] == id]['text'].iloc[0]
        # collecting responses for same question together
        response = assistant_df[assistant_df['parent_id'] == id]['text'].tolist()
        response = ' '.join(response)

        prompts.append(prompt)
        responses.append(response)
        
    # Adding context of first question to all other questions
    prompts = [prompts[0]] + [prompts[0] + element for element in prompts[1:]]
    
    # Storing to temp DF and then concatenating to main output DF
    temp_df = pd.DataFrame({'prompt': prompts, 'response': responses})
    output_df = pd.concat([output_df, temp_df], ignore_index=True)

In [80]:
output_df

,prompt,response
0,Kannst du mir alle Lab Member aus Steins;Gate ...,Alles klar. Hier ist eine Liste der Future Gad...
1,Kannst du mir alle Lab Member aus Steins;Gate ...,
2,¿Para qué sirve la etiqueta <br> de html?,La etiqueta HTML <br> se utiliza para insertar...
3,¿Para qué sirve la etiqueta <br> de html?¿La e...,La etiqueta `<br>` se puede utilizar en la may...
4,¿Para qué sirve la etiqueta <br> de html?¿Es a...,
...,...,...
31520,¿Cómo recuperar una batería de litio que se ha...,
31521,¿Cómo recuperar una batería de litio que se ha...,
31522,¿Cómo recuperar una batería de litio que se ha...,
31523,¿Cómo recuperar una batería de litio que se ha...,


In [82]:
final_df = output_df[output_df['response'] != '']
final_df

,prompt,response
0,Kannst du mir alle Lab Member aus Steins;Gate ...,Alles klar. Hier ist eine Liste der Future Gad...
2,¿Para qué sirve la etiqueta <br> de html?,La etiqueta HTML <br> se utiliza para insertar...
3,¿Para qué sirve la etiqueta <br> de html?¿La e...,La etiqueta `<br>` se puede utilizar en la may...
5,¿Para qué sirve la etiqueta <br> de html?¿Es c...,"No. Es como la tecla ""Intro"" pulsada una sola ..."
6,Cuál es el código para crear un gráfico en Pyt...,Para crear un gráfico en Python con la librerí...
...,...,...
31516,"Мне нужна конфигурация компьютера, на котором ...",Предоставленная конфигурация сможет запустить ...
31517,"Мне нужна конфигурация компьютера, на котором ...",ДОГОВОР НА РАЗРАБОТКУ ВЕБ-САЙТА\n\nМежду _____...
31518,¿Cómo recuperar una batería de litio que se ha...,Lo primero sería asegurarse de que el fallo es...
31519,¿Cómo recuperar una batería de litio que se ha...,"Un placer, cualquier otra consulta estaré aquí..."


In [83]:
# Write DataFrame to a JSON Lines file
with open('finetuning_data.jsonl', 'w') as file:
    for _, row in final_df.iterrows():
        file.write(row.to_json() + '\n')